In [1]:
import sys
from datasets import load_from_disk
from fastNLP import DataSet,Vocabulary,prepare_torch_dataloader
from fastNLP.io import DataBundle

sys.path.append('..')
nerdata = load_from_disk(dataset_path='../peoples_daily_ner')
train_dataset = DataSet.from_pandas(nerdata['train'].to_pandas())
val_dataset = DataSet.from_pandas(nerdata['validation'].to_pandas())
test_dataset = DataSet.from_pandas(nerdata['test'].to_pandas())


datasets = {"train":train_dataset,"val":val_dataset,"test":test_dataset}
data_bundle = DataBundle(datasets=datasets)

/remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_bundle.apply_more(lambda ins:{'words': ins['tokens'], 'seq_len': len(ins['tokens']), 'target': ins['ner_tags']},
                   progress_bar="tqdm")
data_bundle.delete_field('tokens')
data_bundle.delete_field('ner_tags')
data_bundle.delete_field('id')
data_bundle.drop(lambda ins:ins['seq_len'] <= 0) # 清理长度为0 的数据，否则构建词典出问题

In total 3 datasets:
	train has 20864 instances.
	val has 2318 instances.
	test has 4636 instances.

In [3]:
token_vocab = Vocabulary()
token_vocab.from_dataset(data_bundle.get_dataset('train'), field_name='words')
token_vocab.index_dataset(data_bundle.get_dataset('train'), field_name='words')
token_vocab.index_dataset(data_bundle.get_dataset('val'), field_name='words')
token_vocab.index_dataset(data_bundle.get_dataset('test'), field_name='words')
label_vocab = Vocabulary(padding=None, unknown=None)
label_vocab.add_word_lst(['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])

Vocabulary(['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG']...)

In [4]:
train_dataloader = prepare_torch_dataloader(train_dataset, batch_size=16, shuffle=True)
evaluate_dataloader = prepare_torch_dataloader(val_dataset, batch_size=16)
test_dataloader = prepare_torch_dataloader(test_dataset,batch_size=16)

In [5]:
from fastNLP.models.torch import BiLSTMCRF
from fastNLP import Trainer, Evaluator, SpanFPreRecMetric,LoadBestModelCallback, TorchWarmupCallback, Accuracy

callbacks = [
    LoadBestModelCallback(),
    TorchWarmupCallback(),
]

model = BiLSTMCRF(embed=(len(token_vocab), 150), num_classes=len(label_vocab),
                  num_layers=1, hidden_size=150, dropout=0.2)

from torch.optim import AdamW

optimizers = AdamW(params=model.parameters(), lr=1e-3)

In [6]:
deviceId = 7
trainer = Trainer(
    model=model,
    driver='torch',
    device=deviceId,  # 'cuda'
    n_epochs=10,
    optimizers=optimizers,
    train_dataloader=train_dataloader,
    evaluate_dataloaders=evaluate_dataloader,
    metrics={'F1': SpanFPreRecMetric(tag_vocab=label_vocab),'acc':Accuracy()},
    progress_bar = 'tqdm',
    evaluate_every= 200, # -1代表每个epoch一次评估,正数代表batch个批次一次评估
    callbacks=callbacks,
    monitor='f#F1'
    
)

testor = Evaluator(
    model=model,
    driver='torch',
    device = deviceId,
    dataloaders=test_dataloader,
    metrics={'F1': SpanFPreRecMetric(tag_vocab=label_vocab),'acc':Accuracy()}
) 

In [7]:
trainer.run()

[14:23:08] INFO     Running evaluator sanity check for 2 batches.              ]8;id=410024;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=650187;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Epoch:0:   2%|▏         | [00:14<20:01, 121.98s/it, ]     









































[14:23:27] INFO     ++++++++++Eval. results on Epoch:0,              ]8;id=339537;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=636663;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#322\322]8;;\
                    Batch:200++++++++++                                                      

           INFO     {"f#F1": 0.0, "pre#F1": 0.0, "rec#F1": 0.0,      ]8;id=85838;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=663087;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#327\327]8;;\
                    "acc#acc": 0.87041}                                                      

Epoch:0:   3%|▎         | [00:32<13:33, 83.92s/it, ]   









































...

[14:43:22] INFO     ----------Eval. results on Epoch:9,              ]8;id=165286;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=671078;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#322\322]8;;\
                    Batch:1264----------                                                     

           INFO     {"f#F1": 0.827549, "pre#F1": 0.84742, "rec#F1":  ]8;id=771356;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=226050;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#327\327]8;;\
                    0.808589, "acc#acc": 0.977137}                                           

Epoch:9: 100%|█████████▉| [20:17<00:00, 90.60s/it, ]  

[14:43:25] INFO     The best performance for monitor f#F1:0.828571    ]8;id=905818;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=958568;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#37\37]8;;\
                    was achieved in Epoch:8, Global Batch:11600. The                         
                    evaluation result:                                                       
                    {'f#F1': 0.828571, 'pre#F1': 0.853652, 'rec#F1':                         
                    0.804923, 'acc#acc': 0.977473}                                           

           INFO     Loading best model from buffer with f#F1: ]8;id=395087;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py\load_best_model_callback.py]8;;\:]8;id=859184;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py#120\120]8;;\
                    0.828571 (achieved in Epoch: 8, Global                                   
                    Batch: 11600) ...                                                        

In [8]:
trainer.evaluator.run()

{'f#F1': 0.828571, 'pre#F1': 0.853652, 'rec#F1': 0.804923, 'acc#acc': 0.977473}

In [9]:
testor.run()

/remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/rich/live.py:221: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

{'f#F1': 0.831687, 'pre#F1': 0.856259, 'rec#F1': 0.808486, 'acc#acc': 0.977787}

{'f#F1': 0.831687, 'pre#F1': 0.856259, 'rec#F1': 0.808486, 'acc#acc': 0.977787}

In [10]:
import os
 
pid = os.getpid()
!kill -9 $pid

: 

: 